In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

: 

In [2]:
model=load_model("model.h5")

In [3]:
with open("onehot_encoder_geo.pkl","rb") as file:
    label_encoder_geo=pickle.load(file)
 
with open("label_encode_gender.pkl","rb") as file:
    label_encoder_gender=pickle.load(file)
 
with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [4]:
input_data={
    'CreditScore':600,  
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [5]:
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded

c:\Users\HP\OneDrive\Desktop\knn\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [6]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
input_df['Gender']=label_encoder_gender.transform([input_df['Gender']])
input_df

c:\Users\HP\OneDrive\Desktop\knn\myenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.5202592 ,  0.9206384 ,  0.09451246, -0.70388869, -0.27537458,
         0.81118618,  0.64920267,  0.96510833, -0.87591581,  1.0010005 ,
        -0.58408625, -0.57138416]])

In [11]:
model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step


array([[0.02542274]], dtype=float32)

In [12]:
prediction_prob=model.predict(input_scaled)[0][0]
prediction_prob

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


np.float32(0.025422739)

In [13]:
model.layers[0].weights

[<Variable path=sequential/H_1/kernel, shape=(12, 64), dtype=float32, value=[[ 1.15847416e-01  8.59155878e-02 -6.26869723e-02  1.73641831e-01
    9.45352241e-02 -3.88673060e-02 -9.35058594e-02  2.24041909e-01
   -4.77594249e-02  1.55557498e-01 -1.56718820e-01 -1.66944951e-01
    2.33207256e-01 -2.08050445e-01 -3.34040105e-01 -7.72522315e-02
    1.85415030e-01  1.52728766e-01 -1.88985050e-01  4.89596687e-02
    9.61102024e-02  2.67778873e-01  9.93136093e-02 -1.48511678e-01
    1.94520891e-01  8.35264325e-02  2.30627302e-02  5.45219183e-02
    1.20651931e-01  1.70811430e-01 -2.30029766e-02  5.68170920e-02
    2.79984742e-01 -2.45979413e-01  2.06404701e-02 -2.75958419e-01
    4.56325300e-02 -6.63300678e-02 -1.24218985e-01  2.23366991e-01
    1.32516444e-01 -3.04875821e-01 -2.87957311e-01  2.59834200e-01
    1.57056805e-02 -7.25620613e-02  5.55365197e-02 -3.47896926e-02
   -1.33531481e-01 -1.41689569e-01 -1.59628585e-01  4.75779623e-02
    2.56805420e-01 -6.31885976e-02 -3.19540501e-02  2.

In [14]:
if prediction_prob >0.5:
    print("The customer is likely to churn.")
else:
    print("The coustomer is not likely to churn.")


The coustomer is not likely to churn.
